In [2]:
import os
import cv2
import numpy as np
import ipywidgets as ipyw
import lz4.frame
import pandas as pd

import matplotlib

from matplotlib import pyplot as plt
%matplotlib inline

from werdich_cfr.utils.visuals import ImageSliceViewer3D

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [24]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
meta_date = '200208'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+meta_date)
cfr_meta_file = 'tfr_files_dset_BWH_'+meta_date+'.parquet'
meta_df = pd.read_parquet(os.path.join(meta_dir, cfr_meta_file))

# Filter a4c
meta_a4c = meta_df[meta_df.max_view == 'a4c']

# Round the scale factors
meta_a4c = meta_a4c.round({'deltaX': 4, 'deltaY':4})

In [26]:
meta_a4c.head()

,mrn,study,echo_study_date,reportID,days_post_cfr,subjectid,report_number,cfr_study_date,cfr_report_date,cfr,filename,dir,datetime,fileid,institution,model,manufacturer,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view,mode,rate
130118,24970055,4907b0fca0176e93_4903a58584283b8a636f45a3b841,2010-06-15,104029,-8,74,EVS0251812,2010-06-23,2010-06-15,1.279711,4907b0fca0176e93_4903a58584283b8a636f45a3b841_...,/mnt/obi0/phi/echo/npyFiles/BWH/4907/4907b0fca...,2010-06-15 13:07:49,4907b0fca0176e93_4903a58584283b8a636f45a3b841_...,BWH,Vivid E9,GE Vingmed Ultrasound,40.510204,50.0,96.0,0.0443,0.0443,1.656415e-12,1.417382e-13,2.441444e-15,8.360451e-15,1.719732e-14,2.318683e-15,1.000000,7.620976e-12,7.415833e-13,4.434564e-15,8.511697e-15,9.573756e-15,8.805449e-12,3.314062e-13,6.182626e-15,8.831256e-18,2.148646e-14,2.315238e-14,5.013646e-15,1.518011e-14,8.183547e-14,1.131694e-13,2.399734e-16,2.496736e-14,6.676146e-14,3.026207e-17,8.052248e-14,a4c,train,24.7
45753,21084884,49028a12c595f9db_4903a44ab12daef503e3997935ab,2009-06-15,96110,-1,2343,EVS0189282,2009-06-16,2009-06-15,1.883636,49028a12c595f9db_4903a44ab12daef503e3997935ab_...,/mnt/obi0/phi/echo/npyFiles/BWH/4902/49028a12c...,2009-06-15 15:18:42,49028a12c595f9db_4903a44ab12daef503e3997935ab_...,BWH,Vivid7,GE Vingmed Ultrasound,33.484151,90.0,62.0,0.0435,0.0435,4.443658e-15,1.352233e-16,2.101148e-17,6.101796e-21,3.207646e-18,3.267184e-17,1.000000,3.357259e-14,3.529322e-20,1.474131e-16,1.688050e-17,2.644312e-17,3.165970e-20,2.734201e-17,3.557520e-19,3.147046e-17,1.244661e-17,2.861214e-16,1.746608e-20,2.355914e-17,7.942404e-15,3.983735e-18,2.030724e-16,3.729196e-15,7.393548e-20,1.946166e-18,2.187866e-19,a4c,eval,29.9
95245,1737006,4b7a8092756c2634_4903a444c23a2b4220e621c87aae,2007-10-18,73037,-7,1069,0728042D,2007-10-25,2007-10-18,1.451592,4b7a8092756c2634_4903a444c23a2b4220e621c87aae_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7a/4b7a80927...,2007-10-18 09:03:45,4b7a8092756c2634_4903a444c23a2b4220e621c87aae_...,BWH,iE33,Philips Medical Systems,33.333000,93.0,63.0,0.0440,0.0440,1.008175e-12,7.421610e-12,2.416639e-15,8.845460e-16,1.254276e-16,8.685168e-16,1.000000,3.118159e-12,8.220572e-15,1.973346e-13,1.442674e-12,1.546610e-15,7.383879e-13,2.617057e-13,1.689981e-11,4.033147e-14,1.668058e-14,1.539572e-13,2.490745e-16,1.148214e-13,2.541527e-13,4.561939e-19,8.727817e-15,1.790654e-11,7.331187e-16,5.866990e-20,2.717658e-16,a4c,train,30.0
35214,25003559,4906a124ed0d0dce_4903a58584298228583bf8226717,2010-07-28,104724,-2,4287,EVS0258218,2010-07-30,2010-07-28,1.261163,4906a124ed0d0dce_4903a58584298228583bf8226717_...,/mnt/obi0/phi/echo/npyFiles/BWH/4906/4906a124e...,2010-07-28 10:32:05,4906a124ed0d0dce_4903a58584298228583bf8226717_...,BWH,iE33,Philips Medical Systems,33.333000,57.0,52.0,0.0484,0.0484,6.280726e-09,5.618788e-08,1.532429e-09,7.443003e-08,1.936070e-11,2.752803e-10,0.999960,5.006364e-08,4.294121e-11,2.837334e-09,1.130605e-08,6.385825e-10,1.935780e-09,2.326480e-08,8.177478e-10,7.668254e-10,1.549175e-09,1.881452e-11,9.320009e-09,3.179743e-09,1.659007e-09,3.316424e-05,1.893495e-10,6.049284e-06,3.436605e-07,2.568646e-10,3.416052e-07,a4c,train,30.0
2888,6776660,4b7d73d1f269364e_4903a5835836932bc9a51f4c8af5,2016-01-11,138313,280,6130,E1803929,2015-04-06,2016-02-20,2.196591,4b7d73d1f269364e_4903a5835836932bc9a51f4c8af5_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7d/4b7d73d1f...,2016-01-11 08:33:49,4b7d73d1f269364e_4903a5835836932bc9a51f4c8af5_...,BWH,EPIQ 7C,Philips Medical Systems,33.333000,61.0,134.0,0.0386,0.0386,2.537820e-05,3.541981e-05,1.904881e-06,7.207199e-06,1.936495e-05,3.404811e-07,0.992921,2.080941e-07,2.522381e-06,4.264887e-04,1.403350e-03,1.209978e-04,1.459053e-04,7.677718e-06,2.003447e-04,4.967668e-07,3.305628e-07,7.708289e-08,4.362880e-07,3.415568e

In [7]:
np.min(meta_a4c.deltaX)

-0.0399999991

In [8]:
np.max(meta_a4c.deltaX)

1.0

In [15]:
# Load videos

filename = list(df.filename.unique())[100]
ser = df.loc[df.filename == filename, :].iloc[0]
file = os.path.join(ser.dir, filename)

def load_data(file):



try:
    with lz4.frame.open(file, 'rb') as fp:
        data = np.load(fp)

except IOError as err:
    print('Could not open this file: {}\n {}'.format(file, err))

print()
print(file)
print()
print(data.shape)

View:view_a4c, mode:train, min_rate:20, n_videos:10677

/mnt/obi0/phi/echo/npyFiles/BWH/4b72/4b729ab7c8fbe1dc_4903a581a72ff08f5060c835fa63/4b729ab7c8fbe1dc_4903a581a72ff08f5060c835fa63_Image-44.npy.lz4

(89, 600, 800, 1)


In [17]:
def im_scale(im):
    """ convert single images to uint8 and contrast en"""
    # We can do other things here: e.g. background subtraction or contrast enhancement
    im_scaled = np.uint8((im - np.amin(im))/(np.amax(im) - np.amin(im))*256)
    #im_scaled_eq = cv2.equalizeHist(im_scaled)
    return im_scaled

def data2imarray(im_data):
    """
    apply imscale function to np.array
    arg: im_array (frame, height, width)
    returns: im_array (height, width, frame)
    """
    im_data = np.squeeze(im_data)
    im_list = [im_scale(im_data[im]) for im in range(im_data.shape[0])]
    im_array = np.array(im_list, dtype=np.uint16)
    im_array = np.moveaxis(im_array, 0, -1)
    return im_array

im_array = data2imarray(data)
print(im_array.shape)

(600, 800, 89)


In [20]:
# subsampling to have the same frame rate

def subsample_time_index_list(frame_time, default_rate, min_frames):
    """
    rate: data frame rate,
    default_rate: desired frame rate,
    n_frames: number frames in the default rate (30)
    """
    default_times = np.arange(0, min_frames, 1) / default_rate
    times = np.arange(0, default_times[-1] + frame_time, frame_time)
    time_index_list = [np.argmin(np.abs(times - t)) for t in default_times]

    return time_index_list

def subsample_video(image_array, frame_time, min_rate, min_frames):
    """
    Select frames that are closest to a constant frame rate
    arg: image_array: np.array() [rows, columns, frame]
    """
    convert_video = True
    rate = 1 / frame_time
    # Check if the video is long enough
    min_video_len = min_frames / min_rate
    video_len = image_array.shape[-1] / rate
    if (min_video_len <= video_len) & (min_rate < rate):
        # print('Video is long enough and the rate is good.')
        # Get the frame index list
        time_index_list = subsample_time_index_list(frame_time=frame_time,
                                                    default_rate=min_rate,
                                                    min_frames=min_frames)
        # Select the frames from the video
        image_array = image_array[:, :, time_index_list]
    else:
        print('Frame rate: {:.1f} fps, length: {:.1f} s. Skipping.'.format(rate, video_len))
        convert_video = False

    return convert_video, image_array

In [38]:
frame_time

0.033333

In [21]:
min_rate = 20 # Minimum acceptable frame rage [fps]
min_frames = 30 # Minimum number of frames at min_rate (1.5 s)
convert_video, image_array = subsample_video(im_array, 
                                             frame_time = frame_time, 
                                             min_rate = min_rate, 
                                             min_frames = min_frames)
print(convert_video)
print(image_array.shape)

True
(600, 800, 30)


In [37]:
ImageSliceViewer3D(image_array)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…